In [1]:
import models
import text
import utils
import data_utils
import json
import commons
import torch
import numpy as np
from scipy.io.wavfile import write

In [2]:
with open(r'pretrained/config.json', 'r') as f:
    config = json.load(f)

In [3]:
device = 'cpu'

In [4]:
from text.symbols import symbols
net_g = models.SynthesizerTrn(
    len(symbols),
    80,
    config['train']['segment_size'] // config['data']['hop_length'],
    n_speakers=config['data']['n_speakers'],
    mas_noise_scale_initial=0.01,
    noise_scale_delta=2e-6,
    **config['model']).cpu()

256 2
Multi-band iSTFT VITS2


/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [5]:
from train_finetune_QAT_everything_2 import prepare_model_for_qat
net_g = prepare_model_for_qat(net_g)

/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[QAT] Starting QAT fine-tuning of DECODER only (backend=fbgemm)
[QAT] Removing weight norm from FULL net_g...


/home/michael/Documents/ITMO/EDLM/vosk-ft-test/.venv2/lib/python3.12/site-packages/torch/ao/quantization/observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


[QAT] QAT is ENABLED for net_g.dec (convs/convT/linear via QuantWrapper).


In [6]:
# from torch.ao.quantization import convert

# net_g.dec = convert(net_g.dec, inplace=False)

In [7]:
utils.load_checkpoint(r"db-finetune/out/G_260.pth",
                    net_g,
                    None)
net_g.eval()

INFO:root:Loaded checkpoint 'db-finetune/out/G_260.pth' (iteration 20)


SynthesizerTrn(
  (enc_p): TextEncoder(
    (emb): Embedding(62, 192)
    (encoder): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0): MultiHeadAttention(
          (conv_q): QuantWrapper(
            (quant): QuantStub(
              (activation_post_process): FakeQuantize(
                fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=0, quant_max=127, dtype=torch.quint8, qscheme=torch.per_tensor_affine, ch_axis=-1, scale=tensor([0.1033]), zero_point=tensor([70], dtype=torch.int32)
                (activation_post_process): MovingAverageMinMaxObserver(min_val=-7.19466495513916, max_val=5.924840450286865)
              )
            )
            (dequant): DeQuantStub()
            (module): Conv1d(
              192, 192, kernel_size=(1,), stride=(1,)
              (activation_post_process): FakeQuantize(
                fake_quant_enabled=tensor([1], dtype=torch.uint8), o

In [8]:
txt = 'Я текст сгене+рированный мод+е+лью, загруженной из чекп+оинта. Если меня озвучили - отправивший его+++ достоин уважения.'
out = 'congrats_qat_nonquanted'

In [9]:
def get_text(txt, config):
    text_norm = text.text_to_sequence_g2p(txt)
    if config['data']['add_blank']:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    print(text_norm)
    return text_norm

In [10]:
def vcss(out, inputstr, i):  # single
    device = torch.device("cpu")  # ВАЖНО: quantized модель = CPU
    net_g.to(device)
    net_g.eval()

    stn_tst = get_text(inputstr, config)

    speed = 1.0
    output_dir = r'outputs'
    sid = torch.LongTensor([i]).to(device)

    with torch.no_grad():
        x_tst = stn_tst.to(device).unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

        o, o_mb, *_ = net_g.infer(
            x_tst,
            x_tst_lengths,
            sid=sid,
            noise_scale=.667,
            noise_scale_w=0.8,
            length_scale=1 / speed,
        )

        audio = o[0, 0].cpu().numpy() * 32768.0  # vol scale

    write(rf'{output_dir}/{out}.wav', config['data']['sampling_rate'], audio.astype(np.int16))
    print(rf'{out}.wav Generated!')


In [11]:
vcss(out, txt, 1)

tensor([ 1,  0, 32,  0, 14,  0,  3,  0, 52,  0, 23,  0, 33,  0, 47,  0, 51,  0,
         3,  0, 47,  0, 27,  0, 22,  0, 40,  0, 22,  0, 46,  0, 30,  0, 45,  0,
        41,  0, 55,  0, 14,  0, 39,  0, 39,  0, 57,  0, 32,  0,  3,  0, 37,  0,
        41,  0, 21,  0, 23,  0, 36,  0, 32,  0, 53,  0,  8,  0,  3,  0, 59,  0,
        14,  0, 26,  0, 45,  0, 54,  0, 60,  0, 22,  0, 39,  0, 39,  0, 41,  0,
        32,  0,  3,  0, 31,  0, 59,  0,  3,  0, 19,  0, 22,  0, 33,  0, 43,  0,
        42,  0, 30,  0, 39,  0, 51,  0, 14,  0, 10,  0,  3,  0, 32,  0, 23,  0,
        47,  0, 36,  0, 30,  0,  3,  0, 38,  0, 22,  0, 40,  0, 15,  0,  3,  0,
        41,  0, 59,  0, 55,  0, 54,  0, 19,  0, 30,  0, 36,  0, 30,  0,  3,  0,
         9,  0,  3,  0, 41,  0, 51,  0, 43,  0, 45,  0, 15,  0, 56,  0, 30,  0,
        55,  0, 49,  0, 30,  0, 32,  0,  3,  0, 32,  0, 22,  0, 26,  0, 41,  0,
         3,  0, 20,  0, 41,  0, 47,  0, 51,  0, 42,  0, 30,  0, 39,  0,  3,  0,
        53,  0, 55,  0, 14,  0, 60,  0, 